In [3]:
import os
import numpy as np
from numpy import array
import matplotlib
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
from getdist import plots
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# from mockfactory import Catalog
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, TNSTracerPowerSpectrumMultipoles, FOLPSTracerPowerSpectrumMultipoles
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.samplers.emcee import EmceeSampler
from desilike.samples import Profiles, plotting, Chain
from desilike import setup_logging
setup_logging()  # for logging messages

# the cosmology parameters

kmin     = 0.008
kmax     = 0.20
binning  = 0.006
k_ev     = np.arange(kmin, kmax+0.001, binning)
klim     = {ell*2: (kmin,kmax,binning) for ell in range(2)}

redshift    = 1.0
catalogue   = 'fiducial'  # fiducial, Mnu_p, Mnu_ppp -- QUIJOTE catalogue
cosmology   = 'wCDM'
model       = 'FOLPS'
CovRsf      = 100  # -- covariance rescale factor
r_pks       = ['RSD','LRG','QSO']

for r_pk in r_pks:
    chain_fn = f'./sampler_result/{cosmology}/chain_{model}_{r_pk}_V{CovRsf}_z{redshift}.npy'
    emulator_fn = f'./model/emulator_{cosmology}_z{redshift}.npy'
    if not os.path.exists(chain_fn):
        filename = []
        filedir = f'/Users/alain/Desktop/projectNU/main/data/kbin2/{catalogue}/{r_pk}_z{redshift}/npy/'
        for file in os.listdir(filedir):
            filename.append(filedir+file)
        covariance = filedir+'*'
        if model == 'Kaiser':
            theory = KaiserTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
        elif model == 'TNS':
            theory = TNSTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
        elif model == 'FOLPS':
            theory = FOLPSTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
        observable = TracerPowerSpectrumMultipolesObservable(data= filename,
                                                            covariance= covariance,
                                                            klim=klim,
                                                            theory=theory,
                                                            kin=np.arange(0.001,0.35,0.002))
        likelihood = ObservablesGaussianLikelihood(observable, scale_covariance = 1/CovRsf) #
        likelihood()
        sampler = EmceeSampler(likelihood, seed=39, nwalkers=120, save_fn =chain_fn)
        sampler.run(check={'max_eigen_gr': 0.05}, max_iterations = 9001) # save every 300 iterations

[000000.01] [0/1] 07-02 23:06  Emulator                  INFO     Loading ./model/emulator_wCDM_z1.0.npy.
[000000.85] [0/1] 07-02 23:06  TracerPowerSpectrumMultipolesObservable INFO     Loading 100 files [/Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_122_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_101_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_189_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_141_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_180_z1.0.npy].
[000001.01] [0/1] 07-02 23:06  TracerPowerSpectrumMultipolesObservable INFO     Loading 100 files [/Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_100_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_120_z1.0.npy, ..., /Users/alain/Desktop/

/Users/alain/opt/anaconda3/envs/desilike-env/lib/python3.11/site-packages/desilike/samples/diagnostics.py:289: RuntimeWarning: invalid value encountered in divide
  acf /= acf[0]


[000014.82] [0/1] 07-02 23:07  Diagnostics               INFO     - effective sample size = (150 iterations / integrated autocorrelation time) is nan.
[000014.82] [0/1] 07-02 23:07  Diagnostics               INFO     - current mean acceptance rate is 0.215.
[000014.85] [0/1] 07-02 23:07  TaskManager               INFO     Entering TaskManager with 1 workers.
[000022.01] [0/1] 07-02 23:07  Chain                     INFO     Saving ./sampler_result/wCDM/chain_FOLPS_RSD_V100_z1.0.npy.
[000022.06] [0/1] 07-02 23:07  EmceeSampler              INFO     Diagnostics:
[000022.17] [0/1] 07-02 23:07  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 0.761; not < 0.05.
[000022.26] [0/1] 07-02 23:07  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 0.248.
[000022.62] [0/1] 07-02 23:07  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 0.485.
[000022.67] [0/1] 07-02 23:07  Diagnostics               INFO     - max Geweke is 0.342.
[00002